In [1]:
import numpy as np

In [2]:
test_1 = np.load('/content/forecasts.npy',allow_pickle=True)
data = np.load('/content/forecasts.npy', allow_pickle=True)
stocks_data = []
for k, v in data.item().items():
  
  stocks_data.append(v)

stocks_data = np.array(stocks_data)
stocks_data[0][0]

72.83396000707226

In [256]:
def select_best_time(arr):
    n, = np.shape(arr)
    i,j = 0,1
    start = arr[i]
    profit = 0
    
    best = []
    delta = []
    
    while(j < n):
        curr_profit = arr[j] - start
        profit = max(profit,curr_profit)
        start = min(start,arr[j])
        j+=1
        
        best.append(profit)
        delta.append(curr_profit)
    
    return delta


In [257]:

def select_best_strat(arr):
    m,n = np.shape(arr)
    
    best_set = []
    for i in range(m):
        prof = select_best_time(arr[i])
        best_set.append(prof)
    
    bset = np.array(best_set)
    return bset


In [258]:
def create_row(arr):
    n, = np.shape(arr)
    k = n*(n-1)//2
    
    grid = np.zeros((k,1))
    count = 0
    for i in range(n):
        for j in range(i+1,n):
            grid[count] = ((arr[j] - arr[i])/arr[j])
            count+=1
    grid = grid.reshape(1,k)
    return grid

def create_grid(arr):
    m,n = np.shape(arr)
    k = n*(n-1)//2
    delta = []
    for i in range(m):
        delt = create_row(arr[i])
        delta.append(delt)
    
    delta2 = np.array((delta)).reshape(m,k)
    return delta2,n


In [3]:
maxminval = []
for i in range(stocks_data.shape[0]):
  f = (np.max(stocks_data[i]) - np.min(stocks_data[i]))/np.min(stocks_data[i])
  mxd,mnd = np.argmax(stocks_data[i]),np.argmin(stocks_data[i])
  if mnd > mxd:
    continue
  maxminval.append((f,mxd,mnd,i))

print(len(maxminval))
maxminval.sort(key = lambda x:-x[0])

maxminval[:5]

54


[(0.013561250972735995, 4, 0, 69),
 (0.013478657120164843, 3, 2, 92),
 (0.011703781970551629, 4, 0, 6),
 (0.010078636608751631, 4, 0, 29),
 (0.009858212434872426, 2, 1, 17)]

In [299]:
table,num_nodes = create_grid(trial_arr)
#table[12][6] = 1000

In [261]:
t = np.array(np.max(table,axis = 0))
#num_nodes
t

array([0.0081529 , 0.00548132, 0.0114805 , 0.0133798 , 0.00976198,
       0.01067312, 0.01343664, 0.0132994 , 0.01145222, 0.00672661])

In [262]:
adj_graph = np.full((num_nodes,num_nodes),None)
counter = 0
for i in range(num_nodes):
  for j in range(num_nodes):
    if i==j:
      adj_graph[i][j] = 0
    elif j > i:
      continue
    else:
      adj_graph[i][j] = t[counter]
      counter+=1
adj_graph

array([[0, None, None, None, None],
       [0.008152901460079869, 0, None, None, None],
       [0.005481317293174451, 0.011480504980050323, 0, None, None],
       [0.013379804091485321, 0.009761976793854316, 0.010673115216765857,
        0, None],
       [0.013436644940383439, 0.013299399080060472, 0.011452216621287195,
        0.006726614393795924, 0]], dtype=object)

In [4]:
class Node:
  def __init__(self,val,pred,ID,path):
    self.val = val
    self.pred = pred
    self.ID = ID
    self.path = path
nodes = []


In [264]:

for i in range(num_nodes):
  nodes.append(Node(0,list(range(i)),i,[]))
  

In [265]:
gstack = []
stack = []
flag = (0,0)
for i,j in enumerate(nodes):
  row = adj_graph[i]
  for p in j.pred:
    value = nodes[p].val + row[p]
    if value > j.val:
      flag = (p,i)

    j.val = max(j.val,value)
  #print(flag)
  if j == 0:
    j.path = [0]
  else:
    s,e = flag
    j.path = nodes[s].path + [e]
  gstack.append((flag))


In [266]:
#nodes

In [267]:
for k in range(num_nodes):
  print(k)
  print(nodes[k].val,nodes[k].path)

0
0 [0]
1
0.008152901460079869 [0, 1]
2
0.019633406440130192 [0, 1, 2]
3
0.030306521656896047 [0, 1, 2, 3]
4
0.03703313605069197 [0, 1, 2, 3, 4]


In [268]:
# path = []

# for index,val in enumerate(stack):
#   if index == 0:
#     start,finish = val
#     path.append((start,finish))
#     continue
  
#   cur_s,cur_e = val
#   if cur_s < finish:
#     path = []
  
#   start,finish = cur_s,cur_e
#   path.append((start,finish))
p_path = nodes[-1].path

path = list(zip(p_path[:-1],p_path[1:]))
list(path)




[(0, 1), (1, 2), (2, 3), (3, 4)]

In [269]:
if path[0] == (0,0):
  path = path[1:]
path

[(0, 1), (1, 2), (2, 3), (3, 4)]

In [270]:
summer = 0
strat = []
for buy_day,sell_day in path:
  stock_val = adj_graph[sell_day][buy_day]
  print(stock_val)
  stock,delta = np.where(table == stock_val)
  #print(float(stock))
  strat.append(((stock),(buy_day,sell_day)))
strat

0.008152901460079869
0.011480504980050323
0.010673115216765857
0.006726614393795924


[(array([92]), (0, 1)),
 (array([92]), (1, 2)),
 (array([81]), (2, 3)),
 (array([6]), (3, 4))]

In [271]:
# while(delta>num_nodes):
#   delta -= num_nodes
#   num_nodes-=1

#   if num_nodes == 0:
#     break

In [272]:
delta

array([9])

In [273]:
np.argmax(table,axis=0)

array([92, 71, 92, 69, 17, 81, 81, 92, 81,  6])

In [8]:
class Strat_predictor:
  def __init__(self,arr):
    self.data = arr
  
  def create_row(self,arr):
    n, = np.shape(arr)
    k = n*(n-1)//2
    
    grid = np.zeros((k,1))
    count = 0
    for i in range(n):
        for j in range(i+1,n):
            grid[count] = ((arr[j] - arr[i])/arr[j])
            count+=1
    grid = grid.reshape(1,k)
    return grid

  def create_grid(self,arr):
    m,n = np.shape(arr)
    k = n*(n-1)//2
    delta = []
    for i in range(m):
        delt = self.create_row(arr[i])
        delta.append(delt)
    
    delta2 = np.array((delta)).reshape(m,k)
    return delta2,n
  
  def create_matrix(self):
    self.table,num_nodes = self.create_grid(self.data)
    self.t = np.array(np.max(self.table,axis = 0))

    self.adj_graph = np.full((num_nodes,num_nodes),None)
    
    counter = 0
    for i in range(num_nodes):
      for j in range(num_nodes):
        if i==j:
          self.adj_graph[i][j] = 0
        elif j > i:
          continue
        else:
          self.adj_graph[i][j] = self.t[counter]
          counter+=1
    return num_nodes
  
  def setup_graph(self):
    self.num_nodes = self.create_matrix()
    self.nodes = []
    for i in range(self.num_nodes):
      self.nodes.append(Node(0,list(range(i)),i,[]))
  
  def get_strat(self):
    self.setup_graph()
    gstack = []
    stack = []
    flag = (0,0)
    for i,j in enumerate(self.nodes):
      row = self.adj_graph[i]
      for p in j.pred:
        value = self.nodes[p].val + row[p]
        if value > j.val:
          flag = (p,i)

        j.val = max(j.val,value)
      #print(flag)
      if j == 0:
        j.path = [0]
      else:
        s,e = flag
        j.path = self.nodes[s].path + [e]
      gstack.append((flag))

  def check(self):
    self.get_strat()
    best_node_val = 0
    for k in range(self.num_nodes):
      #print(self.nodes[k].val,self.nodes[k].path)
      vals = self.nodes[k].val
      if vals > best_node_val:
        best_node_index = k
      best_node_val = max(best_node_val,vals)
    
    best_node_path = self.nodes[best_node_index].path
    p_path = best_node_path

    path = list(zip(p_path[:-1],p_path[1:]))
    return path
    
  def predict(self,path):
    self.strat = []
    for buy_day,sell_day in path:
      stock_val = self.adj_graph[sell_day][buy_day]
      stock,delta = np.where(self.table == stock_val)
      self.strat.append(((stock),(buy_day,sell_day)))
      #print(self.strat)
    return self.strat

  def display(self):
    money = 1000
    for stock,(b,s) in self.strat:
      print(stock)
      #index = int(stock)
      index = stock[0]
      #print(index)
      print('Buy Stock with ID',index,'at end of day',b,'and sell at end of day',s)
      stock_num = money/self.data[index][b]
      money = stock_num*self.data[index][s]
    print("Profit made on investement of 1000 Dollars is",money-1000)

In [9]:
# h = Strat_predictor(trial_arr)
# y = h.check()
# strat = h.predict(y)
# h.display()

In [10]:
h = Strat_predictor(stocks_data)
y = h.check()
strat = h.predict(y)
h.display()

[92]
Buy Stock with ID 92 at end of day 0 and sell at end of day 1
[92]
Buy Stock with ID 92 at end of day 1 and sell at end of day 2
[81]
Buy Stock with ID 81 at end of day 2 and sell at end of day 3
[6]
Buy Stock with ID 6 at end of day 3 and sell at end of day 4
Profit made on investement of 1000 Dollars is 13.721966577622084


In [11]:
strat

[(array([92]), (0, 1)),
 (array([92]), (1, 2)),
 (array([81]), (2, 3)),
 (array([6]), (3, 4))]

In [12]:
#stocks_data